In [ ]:
import os
import glob
import errno
import random
import urllib
import numpy as np
from scipy.io import loadmat
from requests import get  # to make GET request


class CWRU:

    def __init__(self, exp, rpm, length):
        if exp not in ('12DriveEndFault', '12FanEndFault', '48DriveEndFault'):
            print("wrong experiment name: {}".format(exp))
            exit(1)
        if rpm not in ('1797', '1772', '1750', '1730'):
            print("wrong rpm value: {}".format(rpm))
            exit(1)
        # root directory of all data
        rdir = os.path.join('DataSetCWRU',
                            exp,
                            rpm)

        fmeta = os.path.join('metadata.txt')
        all_lines = open(fmeta).readlines()
        lines = []
        for line in all_lines:
            l = line.split()
            if (l[0] == exp or l[0] == 'NormalBaseline') and l[1] == rpm:
                lines.append(l)

        self.length = length  # sequence length
        self._load_and_slice_data(rdir, lines)
        # shuffle training and test arrays
        #self._shuffle()
        self.labels = tuple(line[2] for line in lines)
        self.nclasses = len(self.labels)  # number of classes

    def _mkdir(self, path):
        try:
            os.makedirs(path)
        except OSError as exc:
            if exc.errno == errno.EEXIST and os.path.isdir(path):
                pass
            else:
                print("can't create directory '{}''".format(path))
                exit(1)

    def _download(self, fpath, link):
        print("Downloading from {} to: '{}'".format(link, fpath))
        with open(fpath, "wb") as file:
            # get request
            response = get(link)
            # write to file
            file.write(response.content)

    def _load_and_slice_data(self, rdir, infos):
        self.X_train = np.zeros((0, self.length))
        self.X_test = np.zeros((0, self.length))
        #self.X_train = []
        #self.X_test = []
        self.y_train = []
        self.y_test = []
        #print(infos)
        for idx, info in enumerate(infos):
            # directory of this file
            fdir = os.path.join(rdir, info[0], info[1])
            self._mkdir(fdir)
            fpath = os.path.join(fdir, info[2] + '.mat')
            if not os.path.exists(fpath):
                self._download(fpath, info[3].rstrip('\n'))

            mat_dict = loadmat(fpath)
            key = list(filter(lambda x: 'DE_time' in x, mat_dict.keys()))[0]
            time_series = mat_dict[key][:, 0]

            idx_last = -(time_series.shape[0] % self.length)
            clips = time_series[:idx_last].reshape(-1, self.length)
            n = clips.shape[0]
            n_split = int(3 * n / 4)
#             temp_X_train = np.zeros((0, self.length))
#             temp_X_test = np.zeros((0, self.length))
            self.X_train = np.vstack((self.X_train, clips[:n_split]))
            self.X_test = np.vstack((self.X_test, clips[n_split:]))
            #self.X_train.append(clips[:n_split])
            #self.X_test.append(clips[n_split:300])
#             self.X_train.append(temp_X_train)
#             self.X_test.append(temp_X_test)
            self.y_train += [idx] * n_split
            self.y_test += [idx] * (clips.shape[0] - n_split)
        #self.X_train = np.dstack(self.X_train)
        #self.y_train = np.dstack(self.y_train)
        #self.X_test = np.dstack(self.X_test)
        #self.y_test = np.dstack(self.y_test)

    def _shuffle(self):
        # shuffle training samples
        index = list(range(self.X_train.shape[0]))
        random.Random(0).shuffle(index)
        self.X_train = self.X_train[index]
        self.y_train = tuple(self.y_train[i] for i in index)

        # shuffle test samples
        index = list(range(self.X_test.shape[0]))
        random.Random(0).shuffle(index)
        self.X_test = self.X_test[index]
        self.y_test = tuple(self.y_test[i] for i in index)

In [ ]:
# Always run from G:\Machine Learning\git_code\time-series\
# Dataset is at G:\Machine Learning\git_code\time-series\DataSetCWRU
data = CWRU("12DriveEndFault", "1772", 384)

In [ ]:
data.X_train = np.array(data.X_train)
data.y_train = np.array(data.y_train)
data.X_test = np.array(data.X_test)
data.y_test = np.array(data.y_test)

In [ ]:
data.X_train.shape, data.y_train.shape, data.X_test.shape, data.y_test.shape

In [ ]:
#data.y_train = data.y_train.reshape(data.y_train.shape[1], data.y_train.shape[0], 1)
#data.y_test = data.y_test.reshape(data.y_test.shape[1], data.y_test.shape[0], 1)

In [ ]:
data.X_train = data.X_train.reshape(data.X_train.shape[0], data.X_train.shape[1], 1)
data.X_test = data.X_test.reshape(data.X_test.shape[0], data.X_test.shape[1], 1)

In [ ]:
data.X_train.shape, data.y_train.shape, data.X_test.shape, data.y_test.shape

In [ ]:
# data.X_train = pd.DataFrame(data.X_train)
# data.y_train = pd.DataFrame(data.y_train)
# data.X_test = pd.DataFrame(data.X_test)
# data.y_test = pd.DataFrame(data.y_test)

In [ ]:
# data.y_train.iloc[36].values

In [ ]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        vy = y.iloc[i + time_steps].values
        ys.append(vy)
    return np.array(Xs), np.array(ys)

In [ ]:
# TIME_STEPS = 12

# # reshape to [samples, time_steps, n_features]

# Xtrain, ytrain = create_dataset(
#   data.X_train,
#   data.y_train,
#   TIME_STEPS
# )

# Xtest, ytest = create_dataset(
#   data.X_test,
#   data.y_test,
#   TIME_STEPS
# )

# print(Xtrain.shape)

In [ ]:
# data.X_train = data.X_train.reshape(data.X_train.shape[0], data.X_train.shape[1], 1)
# data.X_test = data.X_test.reshape(data.X_test.shape[0], data.X_test.shape[1], 1)

In [ ]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot
 
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values
 
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded
 
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(data, prefix=''):
    # load all train
    trainX, trainy = data.X_train, data.y_train
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = data.X_test, data.y_test
    print(testX.shape, testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy
 
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(LSTM(128, input_shape=(n_timesteps,n_features), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(64, input_shape=(n_timesteps,n_features), return_sequences=True))
    model.add(LSTM(32, input_shape=(n_timesteps,n_features)))
    #model.add(Dense(32, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    history = model.fit(trainX, trainy, epochs=epochs, validation_split=0.1, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=1)
    yhat = model.predict(testX, verbose=0)
    return yhat, history, accuracy

# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

# run an experiment
def run_experiment(repeats=1):
    # load data
    trainX, trainy, testX, testy = load_dataset(data)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        yhat, history, score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
    return yhat, history

# run the experiment
yhat, history = run_experiment()

In [ ]:
yh = [np.argmax(y) for y in yhat]

In [ ]:
np.unique(yh)

In [ ]:
yh

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper right')
plt.show()

In [ ]:
# make a prediction
import sys
import itertools

from scipy.stats import randint
from sklearn.metrics import mean_squared_error, r2_score

#yhat = model.predict(data.X_test, verbose=0)
ytest = to_categorical(data.y_test)
yhat.shape, ytest.shape

In [ ]:
np.unique(yh)

In [ ]:
ytestc = np.argmax(ytest, axis=1)
#yhatc = np.argmax(yhat, axis=1)

In [ ]:
#ytest = data.y_test.reshape(data.y_test.shape[0], 1)
rmse = np.sqrt(mean_squared_error(ytestc, yh))
print('Test RMSE: %.3f' % rmse)

In [ ]:
## time steps, every step is one hour (you can easily convert the time step to the actual time index)
## for a demonstration purpose, I only compare the predictions in 200 hours. 
import seaborn as sns
aa=[x for x in range(len(ytest))]
plt.figure(figsize=(20,10))
plt.hist(ytestc, label='actual')
plt.hist(yh, label='prediction')
plt.ylabel('Faults', size=15)
plt.xlabel('Time', size=15)
plt.legend(fontsize=13)
plt.show()